In [1]:
import pandas as pd

CURULES = 7
PRIMER_ESCRUTINIO = pd.read_csv('data/primer_escrutinio_8-2.csv', dtype={ 'Votos': int, 'Partido': int }, low_memory=False)
SEGUNDO_ESCRUTINIO = pd.read_csv('data/segundo_escrutinio_8-2.csv', dtype={ 'Votos': int, 'Partido': int }, low_memory=False)

In [5]:
from escrutinio.cocientes import nominas_con_curul, seleccion_de_curul
from escrutinio.escrutinio_individual import escrutinio_individual
from escrutinio.residuos import residuos

SEGUNDO_ESCRUTINIO['Es R'] = SEGUNDO_ESCRUTINIO['Es R'].apply(lambda x: True if x == 'R' else False)

# Cociente y Mediocociente
nominas_seleccionadas = nominas_con_curul(PRIMER_ESCRUTINIO, CURULES)
diputados_por_cociente_y_mediocociente = seleccion_de_curul(nominas_seleccionadas, SEGUNDO_ESCRUTINIO)

# Escrutinio individual
votos_individuales = escrutinio_individual(SEGUNDO_ESCRUTINIO)

# Residuos
diputados_por_residuo = residuos(votos_individuales, diputados_por_cociente_y_mediocociente, CURULES)
resultado_final = diputados_por_cociente_y_mediocociente + diputados_por_residuo

# Sólo para presentar los resultados
pd.DataFrame([vars(d) for d in resultado_final])

El cociente es 24639, el mediocociente es 12319
Partido RM obtiene cociente
Partido LP2 obtiene cociente
Partido LP2 obtiene cociente
Partido LP2 obtiene cociente
Partido PRD obtiene mediocociente
Partido RM ya tiene curul. No compite por medio cociente.
Partido LP2 ya tiene curul. No compite por medio cociente.


,nombre,partido,votos,tipo
0,Luis Eduardo Camacho,56.0,19267,TipoDeSeleccion.COCIENTE
1,Eduardo Gaitán,58.0,68169,TipoDeSeleccion.COCIENTE
2,Luis Duke,58.0,66324,TipoDeSeleccion.COCIENTE
3,Alexandra Brenes,58.0,65103,TipoDeSeleccion.COCIENTE
4,Raúl Pineda,2.0,15730,TipoDeSeleccion.MEDIOCOCIENTE
5,Doctora Yarelis Rodríguez,58.0,64498,TipoDeSeleccion.RESIDUO
6,Omar Ortega,56.0,17705,TipoDeSeleccion.RESIDUO


In [3]:
from constants.partidos import partidos as PARTIDOS

# Mezclar ambas tablas para mostrar los votos de cada candidato
resultado_final_df = pd.DataFrame([vars(x) for x in resultado_final])
resultado_final_df = resultado_final_df[["nombre", "tipo"]]
votos_individuales_df = pd.DataFrame([vars(x) for x in votos_individuales])
votos_individuales_df = votos_individuales_df[["nombre", "partido", "votos", "is_coalition"]]
tabla = pd.merge(resultado_final_df, votos_individuales_df, on='nombre', how='right')

# Nombres de los partidos
def nombre_del_partido(id: int):
  partido = PARTIDOS.get(id)
  return partido.siglas if partido else id
tabla['partido'] = tabla['partido'].apply(nombre_del_partido)

# Formatear y desplegar la tabla
tabla.sort_values(by=['votos'], ascending=False, inplace=True)
tabla.reset_index(drop=True, inplace=True)
tabla.to_excel('resultados/resultados_8-2.xlsx', index=False)
tabla

,nombre,tipo,partido,votos,is_coalition
0,Eduardo Gaitán,TipoDeSeleccion.COCIENTE,LP2,68169,False
1,Luis Duke,TipoDeSeleccion.COCIENTE,LP2,66324,False
2,Alexandra Brenes,TipoDeSeleccion.COCIENTE,LP2,65103,False
3,Doctora Yarelis Rodríguez,TipoDeSeleccion.RESIDUO,LP2,64498,False
4,Ángel Ortega González,NaN,LP2,64280,False
...,...,...,...,...,...
67,Jovanita de Roa,NaN,ALIANZA,696,False
68,Luis Romero,NaN,ALIANZA,688,False
69,Fernando Samaniego,NaN,ALIANZA,687,False
70,Raúl Patiño,NaN,ALIANZA,674,False
